In [1]:
# 导入:
from sqlalchemy import Column, String, Integer, DateTime
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

# 创建对象的基类:
Base = declarative_base()

# 定义Position对象，表示局面:
class Position(Base):
    # 表的名字:
    __tablename__ = 'position'

    # 表的结构:
    source_file = Column(String, primary_key=True)   # 源文件
    turn = Column(Integer, primary_key=True)         # 回合数
    board = Column(String(180))                      # 盘面
    next_move = Column(String(6))                    # 下一步着法
    next_move_cn = Column(String)                    # 下一步着法中文
    game_time = Column(DateTime)                     # 开始对局的时间，暂时未处理，先保留吧

    def __repr__(self):
        return "<Position(source_file='%s', turn=%d, board='%s', next_move='%s', next_move_cn='%s', game_time='%s')>" % (self.source_file, self.turn, self.board, self.next_move, self.next_move_cn, self.game_time)

In [3]:
import chess as che

# 返回Position数组
def parse_savefupan(file):
    with open(file, 'r') as f:
        # 读取第一行
        line = f.readline()
    raw = line.split(' ')   # 后面有xml，田
    num_list = []           # 去掉xml
    # 找到data中第一个不是数字的位置
    for i in range(len(raw)):
        if raw[i].lstrip('-').isdigit():
            num_list.append(int(raw[i]))
        else:
            break

    # sanity check
    if len(num_list) < 3:
        print(f'棋谱{file}前三位都没有，请检查')
        return None
    length = num_list[1]  # 棋谱长度
    if len(num_list) == 3 + length * 10:
        pass
    elif len(num_list) == 2 + length * 10:
        # 如果最后一步不够10位，补齐，否则恶心
        num_list.append(0)
    else:
        print(f'棋谱{file}长度不对，请检查')
        return None
    
    # 棋谱儿开始了
    board = che.get_standard_init()

    for i in range(3, len(num_list), 10):
        sub_list = num_list[i:i+10]  # 每10位一个子列表
        #3-6位是坐标, [1, 3, 3, 5] 相七进五, [10, 2, 8, 3] 马2进3
        # 坐标转换
        converted = [sub_list[3]-1, sub_list[2]-1, sub_list[5]-1, sub_list[4]-1]
        piece = board[converted[0]][converted[1]]   # 棋子儿
        move = [piece, *converted]              # 着法儿

        pos = Position(
            source_file = file,
            turn = sub_list[7],
            board = che.board_to_string(board),
            next_move = che.move_to_string(move),
            next_move_cn = che.move_get_cn(move, board),
            # game_time = None
        )

        #print(pos)
        print(pos.next_move_cn)

        che.next_board(board, move, replace=True)                 # 更新棋盘

parse_savefupan('../data/savefupan/1e2ad000dc39f60facc9fea212fcf9c0')

相七进五
马2进3
炮二平四
卒3进1
马二进一
马8进7
车一平二
车9平8
车二进六
炮8平9
车二平三
炮9退1
车三退一
士4进5
车三平七
炮9进1
炮八平七
象3进1
车七进一
马3退4
马八进九
马4进5
车七平九
马7进6
后车平八
马6进4
兵七进一
车8进4
兵九进一
卒5进1
车九退一
炮9平7
炮七平六
马5进3
车八进六
炮2平5
车九平六
炮5进4
仕四进五
车1平3
兵七进一
炮7进7
车六退一
炮7平9
帅五平四
马3退5
炮四平三
象7进9
车六平四
车3进4
车八退三
炮5退1
马九进七
车8平6
车四进一
马5进6
马七进五
卒5进1
兵一进一
炮9退4
兵三进一
卒5进1
兵三进一
象9进7
马一进二
卒5进1
炮六平九
马6进4
车八平六
象1退3
马二进一
车3进1
马一退三
炮9平7
炮三平二
象3进5
炮二进七
象5退7
马三进二
炮7平6
马二进三
马4进6
仕五进四
马6退4
帅四平五
炮6平5
帅五平四
士5进4
车六平五
马4进3
车五平六
卒5平6
炮九平四
炮5平6
炮四平五
马3退4
车六平五
将5平4
马三退四
士6进5
马四进三
士5退6
炮五平六
车3进4
车五平六
车3平4
帅四进一
车4退2
车六退一
马4进6
车六平四
炮6进2
帅四进一
